## Train

## Importing Libraries

In [2]:
import pkg_resources

REQUIRED_PACKAGES = [
    'mtcnn','tensorflow,
]

for package in REQUIRED_PACKAGES:
    try:
        dist = pkg_resources.get_distribution(package)
        print('{} ({}) is already installed'.format(dist.key, dist.version))
    except pkg_resources.DistributionNotFound:
        !pip install {package}

SyntaxError: unterminated string literal (detected at line 4) (2467296452.py, line 4)

In [2]:
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN

In [3]:
# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    # bug fix
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

In [4]:
# load images and extract faces for all images in a directory
def load_faces(directory):
    faces = list()
    # enumerate files
    for filename in listdir(directory):
        # path
        path = directory + filename
        # get face
        face = extract_face(path)
        # store
        faces.append(face)
    return faces

In [5]:
# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
    X, y = list(), list()
    # enumerate folders, on per class
    for subdir in listdir(directory):
        # path
        path = directory + subdir + '/'
        # skip any files that might be in the dir
        if not isdir(path):
            continue
        # load all faces in the subdirectory
        faces = load_faces(path)
        # create labels
        labels = [subdir for _ in range(len(faces))]
        # summarize progress
        print('>loaded %d examples for class: %s' % (len(faces), subdir))
        # store
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)

In [6]:
# load train dataset
trainX, trainy = load_dataset('images/train/')
print(trainX.shape, trainy.shape)

# load test dataset
testX, testy = load_dataset('images/val/')

# save arrays to one file in compressed format
savez_compressed('ss.npz', trainX, trainy, testX, testy)

1/1 [==============================] - 0s 186ms/step
>loaded 3 examples for class: 1685_Anjana_Om_Kashyap
1/1 [==============================] - 0s 165ms/step
>loaded 3 examples for class: 1686_Barkha_Dutt
1/1 [==============================] - 0s 174ms/step
>loaded 3 examples for class: 1687_Ravish_Kumar
1/1 [==============================] - 0s 228ms/step
>loaded 4 examples for class: 1688_Arnab_Goswami
(13, 224, 224, 3) (13,)
1/1 [==============================] - 0s 191ms/step
>loaded 1 examples for class: 1685_Anjana_Om_Kashyap
1/1 [==============================] - 0s 174ms/step
>loaded 2 examples for class: 1686_Barkha_Dutt
1/1 [==============================] - 0s 159ms/step
>loaded 2 examples for class: 1687_Ravish_Kumar
1/1 [==============================] - 0s 168ms/step
>loaded 2 examples for class: 1688_Arnab_Goswami


## Embeddings

In [7]:
# calculate a face embedding for each face in the dataset using facenet
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
from keras_facenet import FaceNet
from tensorflow import keras
from sklearn.svm import SVC
from tensorflow import keras
import tensorflow as tf

In [8]:
embedder = FaceNet()

In [10]:
# get the face embedding for one face
def get_embedding(model, face_pixels):
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    detections = model.extract(face_pixels, threshold = 0.95)
    # standardize pixel values across channels (global)
    #mean, std = face_pixels.mean(), face_pixels.std()
    #face_pixels = (face_pixels - mean) / std
    # transform face into one sample
    samples = expand_dims(face_pixels, axis=0)
    # make prediction to get embedding
    # yhat = model.predict(samples)
    yhat = model.embeddings(samples)

    return yhat[0]

In [11]:
# load the face dataset
data = load('ss.npz')

trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)
# load the facenet model
# model = load_model('facenet_keras.h5')
# model.summary()
model = embedder
# model = SVC(kernel='linear', probability=True)
# model = load_model('face-rec_Google.npz')
# model = load('test.npz')
# model = tf.keras.applications.ResNet50(weights='imagenet')

print('Loaded Model')
# convert each face in the train set to an embedding

Loaded:  (13, 224, 224, 3) (13,) (7, 224, 224, 3) (7,)
Loaded Model


In [12]:
newTrainX = list()
for face_pixels in trainX:
    embedding = get_embedding(model, face_pixels)
    newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)
# convert each face in the test set to an embedding
newTestX = list()
for face_pixels in testX:
    embedding = get_embedding(model, face_pixels)
    newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)
# save arrays to one file in compressed format
savez_compressed('ss-embd.npz', newTrainX, trainy, newTestX, testy)

1/1 [==============================] - 0s 73ms/step
(13, 512)
1/1 [==============================] - 0s 74ms/step
(7, 512)


## Webcam

In [13]:
import cv2
import numpy as np
from mtcnn import MTCNN
from keras.models import load_model
from scipy.spatial.distance import cosine
import tensorflow as tf
from keras_facenet import FaceNet

In [14]:
# Load the FaceNet model
facenet_model = FaceNet()

# Load the embeddings from the .npz file
data = np.load('ss-embd.npz')
embeddings_array = data['arr_0']
names = data['arr_1']

# Load the MTCNN face detection model
mtcnn_detector = MTCNN()

In [15]:
# Open the video capture
video_capture = cv2.VideoCapture(0)  # Use 0 for webcam or provide the path to a video file

while True:
    # Read a frame from the video
    ret, frame = video_capture.read()

    if not ret:
        break

    # Detect faces in the frame using MTCNN
    faces = mtcnn_detector.detect_faces(frame)

    # Iterate over detected faces
    for face in faces:
        # Extract the face coordinates
        x, y, w, h = face['box']

        # Extract the face ROI from the frame
        face_roi = frame[y:y+h, x:x+w]

        # Preprocess the face ROI
        face_roi = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
        face_roi = cv2.resize(face_roi, (224, 224))
        face_roi = face_roi.astype('float32')
        # mean, std = face_roi.mean(), face_roi.std()
        # face_roi = (face_roi - mean) / std

        # Convert the preprocessed face ROI to a 4D tensor
        face_tensor = np.expand_dims(face_roi, axis=0)

        # Generate embeddings using the FaceNet model
        face_embedding = facenet_model.embeddings(face_tensor)[0]

        # Compare the face embedding with stored embeddings
        distances = [cosine(face_embedding, emb) for emb in embeddings_array]
        min_distance = min(distances)
        min_distance_idx = np.argmin(distances)

        # Define a threshold for face recognition
        threshold = 0.75

        # Check if the minimum distance is below the threshold
        if min_distance < threshold:
            # Identify the recognized face
            recognized_face = embeddings_array[min_distance_idx]
            recognized_name = names[min_distance_idx]

            # Calculate the confidence level
            confidence = 100 - min_distance*100

            # Draw bounding box and label for the recognized face with confidence level
            label = f"{recognized_name} ({confidence:.2f})"
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        else:
 
            # Draw bounding box and label for unknown face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            cv2.putText(frame, 'Unknown', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # Display the frame
    cv2.imshow('Video', frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all

1/1 [==============================] - 0s 27ms/step


1/1 [==============================] - 0s 65ms/step
